In [1]:
#In the prototype, we focus on "text recognition" only 
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold=KFold(n_splits=10)

In [2]:
# reading in the CSV files as Pandas dataframes
df_train = pd.read_csv('vizwiz_skill_typ_train.csv', skipinitialspace = True, engine='python')
df_val = pd.read_csv('vizwiz_skill_typ_val.csv', skipinitialspace = True, engine='python')
# The structure of VizWiz file
df_train.head(3)

,QID,IMG,QSN,ANS_TYP,TXT,OBJ,COL,CNT,OTH,ANS1,ANS2,ANS3,ANS4,ANS5,ANS6,ANS7,ANS8,ANS9,ANS10
0,VizWiz_train_000000000000.jpg,VizWiz_train_000000000000.jpg,What's the name of this product?,other,1,1,0,0,0,basil leaves,basil leaves,basil,basil,basil leaves,basil leaves,basil leaves,basil leaves,basil leaves,basil
1,VizWiz_train_000000000001.jpg,VizWiz_train_000000000001.jpg,Can you tell me what is in this can please?,other,1,1,0,0,0,soda,coca cola,coca cola,unsuitable,unsuitable,coke 0,coca cola 0,coke 0,coca cola,coke
2,VizWiz_train_000000000002.jpg,VizWiz_train_000000000002.jpg,Is this enchilada sauce or is this tomatoes? ...,other,1,1,0,0,0,these tomatoes not enchilada sauce,tomatoes,tomatoes,tomatoes,tomatoes,crushed tomatoes,crushed tomatoes,tomatoes,tomatoes,tomatoes


In [3]:
#create X and Ys
ques_train=df_train["QSN"].values
txt_train=df_train["TXT"].values
obj_train=df_train["OBJ"].values
col_train=df_train["COL"].values
cnt_train=df_train["CNT"].values

ques_val=df_val["QSN"].values
txt_val=df_val["TXT"].values
obj_val=df_val["OBJ"].values
col_val=df_val["COL"].values
cnt_val=df_val["CNT"].values

In [4]:
#Use LSTM to predict
import gzip
import numpy as np
import os
import pandas as pd
import pickle
import requests
import time

os.environ['KERAS_BACKEND']='cntk'
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.models import Sequential, load_model
from keras import regularizers
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.callbacks import History, CSVLogger
from keras.utils import to_categorical

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using CNTK backend


In [5]:
MAX_DOC_LEN = 40
VOCAB_SIZE = 3000
EMBEDDING_DIM = 100

In [6]:
# tokenize, create seqs, pad
tok = Tokenizer(num_words=VOCAB_SIZE, lower=True, split=" ")
tok.fit_on_texts(ques_train)
train_seq = tok.texts_to_sequences(ques_train)
train_seq = sequence.pad_sequences(train_seq, maxlen=MAX_DOC_LEN)
val_seq = tok.texts_to_sequences(ques_val)
val_seq = sequence.pad_sequences(val_seq, maxlen=MAX_DOC_LEN)

In [7]:
labels = to_categorical(np.asarray(txt_train))
labels = labels.astype('float32')
print('Number of reviews by class in training set')
print(labels.sum(axis=0))
n_classes = labels.shape[1]

Number of reviews by class in training set
[6252. 8017.]


In [8]:
import nltk 

nltk.download('punkt')

sent_lst = []

for doc in ques_train:
    sentences = nltk.tokenize.sent_tokenize(doc)
    for sent in sentences:
        word_lst = [w for w in nltk.tokenize.word_tokenize(sent) if w.isalnum()]
        sent_lst.append(word_lst)

[nltk_data] Downloading package punkt to /home/lovelydf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# use skip-gram
word2vec_model = gensim.models.Word2Vec(sentences=sent_lst, min_count=6, size=EMBEDDING_DIM, sg=1, workers=os.cpu_count())

2018-11-13 07:20:03,691 : INFO : collecting all words and their counts
2018-11-13 07:20:03,692 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-11-13 07:20:03,703 : INFO : PROGRESS: at sentence #10000, processed 54533 words, keeping 2478 word types
2018-11-13 07:20:03,711 : INFO : collected 3155 word types from a corpus of 88953 raw words and 16228 sentences
2018-11-13 07:20:03,712 : INFO : Loading a fresh vocabulary
2018-11-13 07:20:03,715 : INFO : effective_min_count=6 retains 749 unique words (23% of original 3155, drops 2406)
2018-11-13 07:20:03,716 : INFO : effective_min_count=6 leaves 84885 word corpus (95% of original 88953, drops 4068)
2018-11-13 07:20:03,718 : INFO : deleting the raw counts dictionary of 3155 items
2018-11-13 07:20:03,719 : INFO : sample=0.001 downsamples 44 most-common words
2018-11-13 07:20:03,720 : INFO : downsampling leaves estimated 35348 word corpus (41.6% of prior 84885)
2018-11-13 07:20:03,722 : INFO : estimated required 

In [10]:
embeddings_index = {}

for word in word2vec_model.wv.vocab:
    coefs = np.asarray(word2vec_model.wv[word], dtype='float32')
    embeddings_index[word] = coefs

print('Total %s word vectors.' % len(embeddings_index))

# Initial embedding
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

for word, i in tok.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None and i < VOCAB_SIZE:
        embedding_matrix[i] = embedding_vector

Total 749 word vectors.


In [11]:
BATCH_SIZE = 50
NUM_EPOCHS = 30
LSTM_DIM = 100
OPTIMIZER = SGD(lr=0.01, nesterov=True)

In [12]:
def lstm_create_train(reg_param):
    l2_reg = regularizers.l2(reg_param)

    # model init
    embedding_layer = Embedding(VOCAB_SIZE,
                                EMBEDDING_DIM,
                                input_length=MAX_DOC_LEN,
                                trainable=True,
                                mask_zero=False,
                                embeddings_regularizer=l2_reg,
                                weights=[embedding_matrix])

    lstm_layer = LSTM(units=LSTM_DIM, kernel_regularizer=l2_reg)
    dense_layer = Dense(n_classes, activation='softmax', kernel_regularizer=l2_reg)

    model = Sequential()
    model.add(embedding_layer)
    model.add(Bidirectional(lstm_layer))
    model.add(dense_layer)

    model.compile(loss='categorical_crossentropy',
                  optimizer=OPTIMIZER,
                  metrics=['acc'])

    fname = "lstm"
    history = History()
    csv_logger = CSVLogger('./{0}_{1}.log'.format(fname, reg_param),
                           separator=',',
                           append=True)

    t1 = time.time()
    # model fit
    model.fit(train_seq,
              labels.astype('float32'),
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCHS,
              callbacks=[history, csv_logger],
              verbose=2)
    t2 = time.time()

    # save model
    model.save('./{0}_{1}_model.h5'.format(fname, reg_param))
    np.savetxt('./{0}_{1}_time.txt'.format(fname, reg_param), 
               [reg_param, (t2-t1) / 3600])
    with open('./{0}_{1}_history.txt'.format(fname, reg_param), "w") as res_file:
        res_file.write(str(history.history))

In [13]:
lstm_create_train(1e-7)

Epoch 1/30
 - 12s - loss: 0.6838 - acc: 0.5590
Epoch 2/30
 - 11s - loss: 0.6772 - acc: 0.5637
Epoch 3/30
 - 11s - loss: 0.6716 - acc: 0.5721
Epoch 4/30
 - 11s - loss: 0.6656 - acc: 0.5781
Epoch 5/30
 - 11s - loss: 0.6597 - acc: 0.5857
Epoch 6/30
 - 11s - loss: 0.6533 - acc: 0.6024
Epoch 7/30
 - 11s - loss: 0.6468 - acc: 0.6281
Epoch 8/30
 - 11s - loss: 0.6401 - acc: 0.6651
Epoch 9/30
 - 11s - loss: 0.6333 - acc: 0.6729
Epoch 10/30
 - 11s - loss: 0.6261 - acc: 0.6778
Epoch 11/30
 - 11s - loss: 0.6186 - acc: 0.6801
Epoch 12/30
 - 11s - loss: 0.6108 - acc: 0.6841
Epoch 13/30
 - 11s - loss: 0.6027 - acc: 0.6883
Epoch 14/30
 - 11s - loss: 0.5941 - acc: 0.6919
Epoch 15/30
 - 11s - loss: 0.5856 - acc: 0.6934
Epoch 16/30
 - 11s - loss: 0.5774 - acc: 0.6973
Epoch 17/30
 - 11s - loss: 0.5699 - acc: 0.6988
Epoch 18/30
 - 11s - loss: 0.5625 - acc: 0.7031
Epoch 19/30
 - 11s - loss: 0.5557 - acc: 0.7050
Epoch 20/30
 - 11s - loss: 0.5485 - acc: 0.7088
Epoch 21/30
 - 11s - loss: 0.5425 - acc: 0.7125
E

In [14]:
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

model = load_model('./lstm_{0}_model.h5'.format(1e-7))
preds = model.predict(val_seq, verbose=0)
print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
       roc_auc_score(txt_val, preds[:, 1]))))

Accuracy = 0.7721012883163039 	 AUC = 0.8429309852839264
